## Batch Normalization

Ref: <a href='https://arxiv.org/abs/1502.03167'>Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift</a>

배치 정규화(Batch Normalization)는 신경망의 각 층에서 입력 데이터의 분포를 안정화시키는 방법이다.

일반적으로 신경망에서는 학습 과정 중에 가중치와 편향이 업데이트되면서 입력 데이터의 분포가 변화할 수 있는데, 이를 논문에서는 `Internal Covariate Shift`라고 정의한다.

데이터의 분포가 레이어를 거치면서 변화할 경우 후속 레이어가 이와 같이 shifted 된 분포에 적응을 해야하는 문제가 발생하기 때문에 학습 속도가 느려질 수 있으며, 가중치 초기값에 따라 학습 결과가 크게 달라질 수 있다는 문제점을 가지게 된다. 이러한 문제는 층이 깊어질수록 더욱 심각해질 수 있기에 딥러닝 연구에서 매우 중요한 문제 중 하나이다.

배치 정규화는 이러한 문제를 해결하기 위해 각 층의 활성화 함수(activation function)의 입력을 평균과 분산을 사용하여 정규화함으로써 입력의 분포를 안정화시키고 학습속도를 향상시키며, 가중치 초기화에 덜 민감하게 만든다.

메인 아이디어는 각 층의 활성화 값이 정규화되어 특정 범위에 머무르도록 하는 것이며 다음과 같은 단계를 거친다.

1. 배치 평균 및 분산 계산: 각 미니 배치 별로 입력 $x$의 평균 $E[x]$와 분산 $Var[x]$를 계산하고 입력을 정규화한다.
$$ \hat{x}={x-\mathbb{E}[x]\over \sqrt{Var[x]+\epsilon}}$$ 

2. 스케일 및 시프트: 정규화된 입력 $\hat{x}$에 학습 가능한 스케일 파라미터 $\gamma$와 시프트 파라미터 $\beta$를 적용하여 네트워크가 적절한 분포를 학습할 수 있도록 한다.

$$ \hat{x}^{(k)}={x^{(k)}-\mathcal{E}[x^{(k)}]\over \sqrt{Var[x^{(k)}]}}$$

$$y^{(k)} = \gamma^{(k)}\hat{x}^{(k)}+\beta^{(k)}$$

Note: 외부에서 미리 계산된(outside gradient computed) 평균을 사용할 경우 $x$의 변화에 따른 $E[x]$의 변화가 반영되지 않게 되어 신경망에서 편향 등의 파라미터가 적절히 조정되지 않을 수 있다. 따라서 배치 정규화에서는 미니 배치 내에서 계산된 평균을 사용한다.

추론 시 활용되는 데이터셋에 대해 평균과 분산을 계산할 수도 있고, 학습 과정 중에 계산된 값을 활용할 수도 있다. 일반적인 경향은 훈련 단계 중 이동지수평균(exponential moving average)을 이용하여 평균과 분산을 계산 후 추론 시 활용하는 것이다.

In [3]:
import torch
from torch import nn

$X\in \mathbb{R}^{B\times C\times H\times W}$를 입력으로 받는 Batch Normalization Layer는 X를 다음과 같이 정규화한다. (B는 배치사이즈, C는 채널의 수, H는 height, W는 width)

$$
BN(X) = \gamma {X-\mathbb{E}_{B,H,W}[X] \over \sqrt{{Var}_{B,H,W}[X] + \epsilon}} + \beta
$$

\* C는 $\gamma$와 $\beta$의 차원이 되며 나머지 값(B,H,W)에 대해서 평균과 분산이 계산됨

$$
if~X\in \mathbb{R}^{B\times C\times L},~\gamma\in\mathbb{R}^C,~\beta\in\mathbb{\R}^C~and~\mathbb{E}_{B,L}[X],~{Var}_{B,L}[X]
$$

In [5]:
class BatchNorm(nn.Module):
    '''
    channels: 입력의 피쳐 수
    eps: 분모에 사용되는 엡실론. div-by-zero 에러 방지
    momentum: 이동지수평균을 위한 모멘텀
    affine: 정규화된 값에 대해 스케일 및 시프트 적용 여부 결정
    track_running_stats: 평군과 분산에 대한 이동평균 계산 여부 결정
    '''
    def __init__(self, channels: int, *,
                 eps: float = 1e-5, momentum: float = 0.1,
                 affine: bool = True, track_running_stats: bool = True):
        super().__init__()
        
        self.channels = channels
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.track_running_stats = track_running_stats
        
        # parameters for scale and shift
        if self.affine:
            self.gamma = nn.Parameter(torch.ones(channels)) # 곱하기 1
            self.beta = nn.Parameter(torch.zeros(channels)) # 더하기 0
        
        # parameters for exponential moving average
        if self.track_running_stats:
            self.register_buffer('exp_mean', torch.zeros(channels)) # 평균 0
            self.register_buffer('exp_var', torch.ones(channels)) # 분산 1
    
    def forward(self, x: torch.Tensor):
        # shape(x) = (batch_size, channels, dimensions*)
        # ex - 2D image x will be (batch_size, channels, height, width)
        
        x_shape = x.shape
        batch_size = x_shape[0]
        
        assert self.channels == x_shape[1]
        
        # reshape into [batch_size, channels, n]
        x = x.view(batch_size, self.channels, -1)
        
        # calculate batch mean & var in training mode,
        # or if we have not tracked exponential moving average
        if self.training or not self.track_running_stats:
            mean = x.mean(dim=[0,2]) # first and last dimension
            mean_x2 = (x ** 2).mean(dim=[0,2])
            
            var = mean_x2 - (mean ** 2)
        
        # update exponential moving averages
        if self.training and self.track_running_stats:
            self.exp_mean = (1-self.momentum) * self.exp_mean + self.momentum * mean
            self.exp_var = (1-self.momentum) * self.exp_var + self.momentum * var
        else:
            # use exponential moving averages as estimates
            mean = self.exp_mean
            var = self.exp_var
            
        # normalize
        x_norm = (x - mean.view(1,-1,1)) / torch.sqrt(var + self.eps).view(1,-1,1)
        
        # scale and shift
        if self.affine:
            x_norm = self.gamma.view(1,-1,1) * x_norm + self.beta.view(1,-1,1)
        
        return x_norm.view(x_shape) # reshape to original shape

In [39]:
def _test():
    x = torch.randn([2,3,2,4]) # B C H W
    print(f'original input: {x})')
    
    
    bn = BatchNorm(3) # channel = 3
    x = bn(x)
    print(f'normalized input: {x}')
    print(f'{bn.exp_mean = }')
    print(f'{bn.exp_var = }')

In [40]:
if __name__ == '__main__':
    _test()

original input: tensor([[[[ 0.5892,  0.1462,  1.3988,  0.1927],
          [-1.0795, -1.0181, -1.0973,  1.6883]],

         [[-0.1495,  0.8476,  0.6319, -0.1521],
          [ 0.8162,  0.3071, -1.1146,  0.2999]],

         [[-0.1654, -0.9820,  0.4337, -0.0719],
          [ 0.2602, -0.2326, -1.0214, -0.2457]]],


        [[[-2.2961, -2.1283,  1.1741, -0.2161],
          [ 0.2236, -0.4049, -0.7777, -0.6352]],

         [[-1.5205,  0.0137,  0.3692, -1.3053],
          [ 0.5722,  0.4965,  0.1500, -0.8167]],

         [[-0.4041,  0.8192, -0.6540,  1.6048],
          [-0.2825,  1.2179, -2.2571, -0.2425]]]]))
normalized input: tensor([[[[ 0.7651,  0.3683,  1.4903,  0.4100],
          [-0.7295, -0.6745, -0.7454,  1.7495]],

         [[-0.1561,  1.1998,  0.9064, -0.1597],
          [ 1.1570,  0.4647, -1.4686,  0.4549]],

         [[-0.0297, -0.9479,  0.6439,  0.0754],
          [ 0.4489, -0.1052, -0.9922, -0.1200]]],


        [[[-1.8192, -1.6689,  1.2890,  0.0438],
          [ 0.4377, -0.1253, -